In [325]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [326]:
import numpy as np # 행렬이나 일반적으로 대규모 다차원 배열을 쉽게 처리할 수 있도록 지원하는 파이썬의 라이브러리이다.
import pandas as pd # 파이썬 데이터 처리를 위한 라이브러리, 특히 숫자테이블과 시계열을 조작하기 위한 데이터 구조와 연산을 제공.
import matplotlib.pyplot as plt # 파이썬 및 숫자 확장 넘파이를 활용한 플로팅 라이브러리, 객체 지향 API를 제공, MATLAB에 대한 실행 가능한 오픈소스 대안을 제공.
import seaborn as sns # Matplotlib을 기반으로 다양한 색상 테마와 통계용 차트 등의 기능을 추가한 시각화 패키지이다. 기본적인 시각화 기능은 Matplotlib 패키지에 의존하며 통계기능은 Statsmodels 패키지에 의존한다.
# seaborn 과 matplotlib은 시각화 단계
plt.style.use('seaborn')
sns.set(font_scale=2.5) # 이 두줄은 본 필자가 항상 쓰는 방법 입니다. matplotlib 의 기본 scheme 말고 seaborn scheme 을 세팅하고, 일일이 graph 의 font size를 지정할 필요 없이 seaborn의 font_scale 을 사용하면 편합니다.
# scheme란 스킴이라고 하며 스키마와 혼동할수 있으나 스키마는 대략적인 계획이나 도식을 뜻하는 데 비해 스킴은 구체적이고 확정된 것을 말한다.
import missingno as msno

#ignore warnings
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

* 앞으로 우리가 해야할 프로세스는 대략 아래와 같습니다
  1. 데이터셋 확인 - 대부분의 캐글 테이터들은 잘 정제되어 있습니다. 하지만 가끔 null data가 존재합니다. 이를 확인하고, 향후 수정합니다.
  2. 탐색적 데이터 분석(exploratory data analysis) - 여러 feature 들을 개별적으로 분석하고, feature 들 간의 상관관계를 확인합니      다.
  3. feature engineering - 모델을 세우기에 앞서, 모델의 성능을 높일 수 잇도록 feature 들을 engineering 합니다.
     one-hot encoding, class로 나누기, 구간으로 나누기, 텍스트 데이터 처리 등을 합니다.
  4. model 만들기 - sklearn 을 사용해 모델을 만듭니다. 파이썬에서 머신러닝을 할 때는 sklearn을 사용하면 수많은 알고리즘을 일관된 문법으      로 사용할수 있습니다. 물론 딥러닝을 위해 tensorflow, pytorch 등을 사용할 수 도 있습니다. (sklearn -> scikit-learn으로 python을 대표하는 머신러닝 라이브러리. 머신러닝 모델에 대해 설명

# 1. Dataset 확인
  - 파이썬에서 테이블화 된 데이터를 다루는 데 가장 최적화되어 있으며, 많이 쓰는 라이브러리는 pandas 입니다.
  - 우리는 pandas 를 사용하여 데이터셋의 간단한 통계적 분석부터, 복잡한 처리들을 간단한 메소드를 통해 사용하여 해낼 수 있습니다.
  - 파이썬으로 데이터 분석을 한다고 하면 반드시 능훅해져야 할 라이브러리이니, 여러 커널들을 반복 또 반복하시길 권장합니다.
  - 캐글에서 데이터셋은 보통 train, testset 으로 나뉘어 있습니다

In [327]:
df_train = pd.read_csv('../input/titanic/train.csv')
df_test = pd.read_csv('../input/titanic/test.csv')

In [328]:
df_train.head()

* 우리가 다루는 문제에서 feature는 pclass, age, SibSp, Parch, Fare 이며, 예측하려는 target label 은 Survived 입니다.

In [329]:
df_train.describe()

In [330]:
df_test.describe()

* 테이블에서 보다시피, PassenserID 숫자와 다른, 그러니깐 null data가 존재하는 열(feature)가 있는 것 같습니다. 
* 이를 좀더 보기 편하기 그래프로 시각화해서 살펴봅시다

* train 과 test로 분리하는 이유 
데이터를 분석한다는 것은 보통 데이터가 주어지면 모델링을 통해 특정 규칙을 찾아 output을 예측하는 것이 목표. 가장 적절한 모델을 찾기 위해 데이터를 train data 와 test data 로 나눈 뒤 train data에 각각의 모델로 학습시켜 test data로 각 모델의 최종 정확도를 확인하기위해서 분리함
* 그렇다면 왜 굳이 train 과 test로 나누는건가?
우리가 궁극적으로 할 것은 unseen data를 예측 혹은 분류하는 것인데 unseen data는 말 그대로 아직 일어나지 않은일의 데이터로 존재하지 않음
그래서 현재 가지고 있는 data를 trainrhk test 로 나눠서 train으로만 학습하면서 최적의 매개변수를 찾고, 그런 다음 Test를 사용하여 모델의 실력을 평가하는 것입니다. (ex. 정확도, MSE...)

# 1.1 Null data check


In [331]:
for col in df_train.columns: #.columns 는 데이터프레임 열의 이름을 조회하기 위한 용도로 쓰여졌다.
    msg = 'column: {:>10}\t Percent of NaN value: {:.2f}%'.format(col, 100 * (df_train[col].isnull().sum( ) / df_train[col].shape[0]))
    print(msg)
    # .format 은 포메팅 함수. 문자열을 이쁘게 만드는 용도 구조 '{인덱스0},{인덱스1}'.format(값0,값1)
    # isnull 결측치 행 확인 메소드. 각 원소의 NaN 값 여부가 표시

In [332]:
for col in df_test.columns:
    msg = 'column {:>10}\t Percent of NaN value: {:2f}%'.format(col, 100 * (df_test[col].isnull().sum() / df_test[col].shape[0]))
    print(msg)

* Train, Test set 에서 Age(둘다 약 20%), Cabin(둘다 약 80%), Embarked(Train만 0.22%) null data 존재하는 것을 볼 수 있습니다.
* MANO 라는 라이브러리를 사용하면 null data의 존재를 더 쉽게 볼 수 있습니다.

In [333]:
msno.matrix(df=df_train.iloc[:, :], figsize=(8, 8), color=(0.8, 0.5, 0.2)) # 여기서도 msno.matrix 가 뭐고 어떤 역할인지 찾기 

In [334]:
msno.bar(df=df_train.iloc[:, :], figsize=(8, 8), color=(0.8, 0.5, 0.2)) # 여기 msno.bar 이게 뭔지 찾기

In [335]:
msno.bar(df=df_test.iloc[:, :], figsize=(8, 8), color=(0.8, 0.5, 0.2))

# 1.2 Target label 확인
* target label 이 어떤 distribution 을 가지고 있는지 지금 확인해봐야 합니다.
* 지금 같은 binary classification 문제의 경우에서, 1과 0의 분포가 어떠냐에 따라 모델의 평가 방법이 달라 질 수 있습니다.

In [336]:
f, ax = plt.subplots(1, 2, figsize=(18, 8))

df_train['Survived'].value_counts().plot.pie(explode=[0, 0.1], autopct='%1.1f%%', ax=ax[0], shadow=True) # pi위에 표시될 글자 형태, 또한 알아서 %로 변환해서 알려줌
ax[0].set_title('Pie plot - Survived')
ax[0].set_ylabel('')
sns.countplot('Survived', data=df_train, ax=ax[1])
ax[1].set_title('Count plot - Survived')

plt.show()

* 안타깝게도 죽은 사람이 많습니다
* 38.4 % 가 살아남았습니다.
* target label 의 분포가 제법 균일(balanced)합니다. 불균일한 경우, 예를 들어서 100중 1이 99, 0이 1개인 경우에는 만약 모델이 모든 것을 1이라 해도 정확도가 99%가 나오게 됩니다. 0을 찾는 문제라면 이 모델은 원하는 결과를 줄 수 없게 됩니다. 지금 문제에서는 그렇지 않으니 계속 진행하겠습니다.

# 2. Exploratory data analysis
* 이제 본격적으로 데이터분석을 해보겠습니다. 데이터는 매우 많습니다. 이 많은 데이터 안에 숨겨진 사실을 찾기 위해선 적절한 시각화가 필요합니다.
* 시각화 라이브러리는 matplotlib, seaborn, plotly 등이 있습니다. 특정 목적에 맞는 소스코드를 정리해두어 필요할때마다 참고하면 편합니다.
## 2.1 pclass
* 먼저 Pclass에 대해서 살펴보겠습니다. Pclass는 ordinal, 서수형 데이터입니다. 카테고리이면서, 순서가 있는 데이터 타입입니다.
* 먼저 Pclass에 따른 생존률 차이를 살펴보겠습니다. 엑셀의 피벗 차트와 유사한 작업을 하게 되는데, pandas dataframe에서는 groupby를 사용하면 쉽게 할 수 있습니다. 또한 pivot 이라는 메소드도 있습니다.
*  'Pclass', 'Survived'를 가져온 후, pclass 로 묶습니다. 그러고 나면 각 pclass 마다 0,1 이 count가 되는데,이를 평균내면 각 pclass 별 생존률이 나옵니다.
* 아래와 같이 count()를 하면, 각 class에 몇명이 있는지 확인할 수 잇으며, sum() 을 하면, 216명중 생존한 (survived=1)사람의 총합을 주게 됩니다.

In [337]:
df_train[['Pclass', 'Survived']].groupby(['Pclass'],as_index=True).count() #groupby()는 연산자로 그룹별로 데이터를 집계, 요약하는 용도. groupby를 쓰면 기본적으로 그룹라벨이 index가 되는데 index를 사용하고 싶지 않을 떄는 as_indes=False를 설정하면 됨

In [338]:
df_train[['Pclass', 'Survived']].groupby(['Pclass'],as_index=True).sum()

* pandas의 crosstab을 사용하면 좀 더 위 과정을 좀 더 수월하게 볼 수 있습니다.

In [339]:
pd.crosstab(df_train['Pclass'], df_train['Survived'],margins=True).style.background_gradient(cmap='summer_r') #crosstab은 pivot table의 일종으로 두 column의 교차 빈도, 비율, 덧셈 등을 구할때 주로 사용 
# cmap은 map에 색깔을 나오게 함

* grouped 객체에 mean()을 하게 되면, 각 클래스별 생존률을 얻을 수 있습니다. class 1 이면 아래와 같습니다.$80\over(80+136)$ $\approx 0.63$

In [340]:
df_train[['Pclass', 'Survived']].groupby(['Pclass'], as_index=True).mean().sort_values(by='Survived', ascending=False).plot.bar()

* 보다시피, Pclass 가 좋을 수록(1st) 생존률이 높은 것을 확인할 수 있습니다.
* 좀 더 보기 쉽게 그래프를 그려보겠습니다. seaborn의 countplot을 이용하면, 특정 label에 따른 개수를 확인해볼 수 있습니다.

In [341]:
y_position = 1.02
f, ax = plt.subplots(1, 2, figsize=(18, 8))
df_train['Pclass'].value_counts().plot.bar(color=['#CD7F32','#FFDF00','#D3D3D3'], ax=ax[0])
ax[0].set_ylabel('Count')
sns.countplot('Pclass', hue='Survived', data=df_train, ax=ax[1])
ax[1].set_title('Pclass : Survived vs Dead', y=y_position)

* '''클래스가 높을 수록, 생존확률이 높은걸 확인할 수 있습니다. Pclass 1, 2, 3 순서대로 63%, 48%, 25% 입니다.'''
* 우리는 생존에 Pclass가 큰 영향을 미친다고 생각해볼 수 있으며, 나중에 모델을 세울 때 이 feature 를 사용해보는 것이 좋을 것이라 판단할 수 있습니다.

# Sex
*     이번에는 성별로 생존률이 어떻게 달라지는 지 확인해보겠습니다.
*    마찬가지로 pandas groupby 와 seaborn countplot 을 사용해서 시각화해봅시다.

In [342]:
f, ax = plt.subplots(1, 2, figsize=(18, 8))
df_train[['Sex','Survived']].groupby(['Sex'], as_index=True).mean().plot.bar(ax=ax[0])
ax[0].set_title('Survived vs Sex')
sns.countplot('Sex', hue='Survived', data=df_train, ax=ax[1])
ax[1].set_title('Sex: survived vs Dead')
plt.show()

* 보시다시피, 여자가 생존할 확률이 높습니다.

In [343]:
df_train[['Sex', 'Survived']].groupby(['Sex'], as_index=False).mean().sort_values(by='Survived', ascending=False)

In [344]:
pd.crosstab(df_train['Sex'], df_train['Survived'], margins=True).style.background_gradient(cmap='summer_r')

* Pclass와 마찬가지로, Sex도 예측 모델에 쓰일 중요한 feature임을 알 수 있습니다

# 2.3 Both Sex and Pclass
*   이번에는 Sex, Pclass 두가지에 관하여 생존이 어떻게 달라지는 지 확인해 봅시다
*   seaborn 의 factorplot을 이용하면, 손쉽게 3개의 차원으로 이루어진 그래프를 그릴 수 있습니다.

In [345]:
sns.factorplot('Pclass', 'Survived', hue='Sex', data=df_train,                            size=6, aspect=1.5) # 여기에는 왜 코드를 중간에 띄워둔걸까

*     모든 클래스에서 female 이 살 확률이 male 보다 높은 걸 알 수 있습니다.
*     또한 남자, 여자 상관없이 클래스가 높을 수록 살 확률이 높습니다.
*     위 그래프는 hue 대신 column으로 하면 아래와 같아집니다.

In [346]:
sns.factorplot(x='Sex', y='Survived', col='Pclass', data=df_train, satureation=5, size = 9, axpect = 1)

# 2.4 Age
*    이번에는 Age feature를 살펴봅시다.

In [347]:
print('제일 나이 많은 탑승객 : {:.1f}) Years'.format(df_train['Age'].max()))
print('제일 어린 탑승객 : {:.1f} Years'.format(df_train['Age'].min()))
print('탑승객 평균 나이 : {:.1f} Years'.format(df_train['Age'].mean()))

* 생존에 따른 Age의 histogram을 그려보겠습니다.

In [348]:
fig, ax = plt.subplots(1, 1, figsize=(9, 5)) #fig는 figure로써 전체 subplot을 말한다-> 전체사이즈 ax는 axe로써 - 전체 중 낱낱개를 말한다. 
sns.kdeplot(df_train[df_train['Survived'] == 1]['Age'], ax=ax)
sns.kdeplot(df_train[df_train['Survived'] == 0]['Age'], ax=ax)
plt.legend(['Survived == 1', 'Survived == 0'])
plt.show()

* 보시다시피, 생존자 중 나이가 어린 경우가 많음을 볼 수 있습니다.

In [349]:
# Age distribution withing classes
plt.figure(figsize=(8, 6)) # figsize는 그래프의 사이즈를 설정하는 용도
df_train['Age'][df_train['Pclass']==1].plot(kind='kde')
df_train['Age'][df_train['Pclass']==2].plot(kind='kde')
df_train['Age'][df_train['Pclass']==3].plot(kind='kde')

plt.xlabel('Age')
plt.title('Age Distribution within classes')
plt.legend(['1st Class','2nd Class', '3rd Class'])

* class 가 높을수록 나이 많은 사람의 비중이 커짐
* 나이대가 변하면서 생존률이 어떻게 되는지 보려고 합니다.
* 나이범위를 점점 넓혀가며, 생존률이 어떻게 되는지 한번 봅시다.

In [350]:
cummulate_survival_ratio = []
for i in range(1, 80):
    cummulate_survival_ratio.append(df_train[df_train['Age']<i]['Survived'].sum() / len(df_train[df_train['Age']<i]['Survived']))

plt.figure(figsize=(7, 7))
plt.plot(cummulate_survival_ratio)
plt.title('Survival rate change depending on range of Age', y=1.02)
plt.ylabel('Survival rate')
plt.xlabel('Range of Age(0~x)')
plt.show()

* 보시다시피, 나이가 어릴 수록 생존률이 확실히 높은 것을 확인할 수 있습니다.
* 우리는 이 나이가 중요한 feature로 쓰일 수 있음을 확인했습니다.

# 2.5 Pclass, Sex, Age
*  지금까지 본, Sex, Pclass, Age, Survived 모두에 대해서 보고싶습니다. 이를 쉽게 그려주는 것이 seaborn의 violinplot 입니다.
*  x축은 우리가 나눠서 보고싶어하는 case(여기선 Pclass, distribution(Age) 입니다.
*  한번 그려보겠습니다.

In [351]:
f, ax=plt.subplots(1,2,figsize=(18, 8))
sns.violinplot("Pclass","Age", hue="Survived", data=df_train, scale='count', split=True,ax=ax[0])
ax[0].set_title('Pclass and Age vs Survived')
ax[0].set_yticks(range(0,110,10))
sns.violinplot("Sex","Age",hue="Survived", data=df_train, scale='count', split=True, ax=ax[1])
ax[1].set_title('Sex and Age vs Survived')
ax[1].set_yticks(range(0,110,10))
plt.show()

* 왼쪽 그림은 Pclass 별로 Age의 distribution 이 어떻게 다른지, 거기에 생존여부에 따라 구분한 그래프 입니다.
* 오른쪽 그림도 마찬가지 Sex, 생존에 따른 distribution이 어떻게 다른지 보여주는 그래프 입니다.
* 생존만 봤을 때, 모든 클래스에서 나이가 어릴수록 생존을 많이 한 것을 볼 수 있습니다.
* 오른쪽 그림에서 보면, 명확히 여자가 생존을 많이 한 것을 볼 수 있습니다.
* 여성과 아이를 먼저 챙긴 것을 볼 수 있습니다.

# 2.6 Embarked
*  Embarked는 탑승한 항구를 나타냅니다.
*  위에서 해왔던 것과 비슷하게 탑승한 곳에 따른 생존률을 보겠습니다.

In [352]:
f, ax = plt.subplots(1, 1, figsize=(7, 7))
df_train[['Embarked', 'Survived']].groupby(['Embarked'], as_index=True).mean().sort_values(by='Survived',ascending=False).plot.bar(ax=ax)

* 보시다시피, 조금의 차이는 있지만 생존률은 좀 비슷한 거 같습니다. 그래도 C가 제일 높군요.
* 모델에 얼마나 큰 영향을 미칠지는 모르겠지만, 그래도 사용하겠습니다.
* 사실, 모델을 만들고 나면 우리가 사용한 feature 들이 얼마나 중요한 역할을 했는지 확인해볼 수 있습니다. 이는 추후에 모델을 만들고 난 다음에 살펴볼 것입니다
* 다른 feature로 split하여 한번 살펴보겠습니다.

In [353]:
f,ax=plt.subplots(2,2,figsize=(20, 15))
sns.countplot('Embarked', data=df_train, ax=ax[0,0])
ax[0,0].set_title('(1)No. Of Passengers Boarded')
sns.countplot('Embarked',hue='Sex',data=df_train, ax=ax[0,1])
ax[0,1].set_title('(2) Male-Female Split for Embarked')
sns.countplot('Embarked', hue='Survived',data=df_train, ax=ax[1,0])
ax[1,0].set_title('(3) Embarked vs Survived')
sns.countplot('Embarked', hue='Pclass', data=df_train, ax=ax[1,1])
ax[1,1].set_title('(4) Embarked vs Pclass')
plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

* Figure(1) - 전체적으로 봤을 때, S에서 가장 많은 사람이 탑승했습니다.
* Figure(2) - C와 Q는 남녀의 비율이 비슷하고, S는 남자가 더 많습니다.
* Figure(3) - 생존확률이 S 경우 많이 낮은 걸 볼 수 있습니다.
* Fiugre(4) - Class로 split해서 보니, c가 생존확률이 높은건 클래스가 높은 사람이 많이 타서 그렇습니다. s 는 3rd class가 많아서 생존확률이 낮게 나옵니다.

# 2.7 Family - Sibsp(형제 자매) + Parch(부모, 자녀)
*  SibSp와 Parch를 합하면 Family가 될 것입니다.
*  Family 로 합쳐서 분석해봅시다.

In [354]:
df_train['FamilySize'] = df_train['SibSp'] + df_train['Parch'] + 1 # 자신을 포함해야하니 1을 더합니다.
df_test['FamilySize'] = df_test['SibSp'] + df_test['Parch'] + 1 # 자신을 포함해야하니 1을 더합니다.

In [355]:
print("Maximum size of Family: ", df_train['FamilySize'].max())
print("Minimum size of Family: ", df_train['FamilySize'].min())

* FamilySize 와 생존의 관계를 한번 살펴봅시다.

In [356]:
f,ax = plt.subplots(1, 3, figsize=(40,10))
sns.countplot('FamilySize', data=df_train, ax=ax[0])
ax[0].set_title('(1) No. Of Passengers Boarded', y=1.02)

sns.countplot('FamilySize', hue='Survived', data=df_train, ax=ax[1])
ax[1].set_title('(2) Survived countplot depending on FamilySize',  y=1.02)

df_train[['FamilySize', 'Survived']].groupby(['FamilySize'], as_index=True).mean().sort_values(by='Survived', ascending=False).plot.bar(ax=ax[2])
ax[2].set_title('(3) Survived rate depending on FamilySize',  y=1.02)

plt.subplots_adjust(wspace=0.2, hspace=0.5)
plt.show()

* Figure (1) - 가족크기가 1~11까지 있음을 볼 수 있습니다. 대부분 1명이고 그 다음으로 2, 3, 4명입니다.
* Figure (2), (3) - 가족 크기에 따른 생존비교입니다. 가족이 4명인 경우가 가장 생존확률이 높습니다. 가족수가 많아질수록, (5, 6, 7, 8, 11) 생존확률이 낮아지네요. 가족수가 너무 작아도(1), 너무 커도(5, 6, 8, 11) 생존 확률이 작네요. 3~4명 선에서 생존확률이 높은 걸 확인할 수 있습니다.


# 2.8 Fare
*  Fare는  탑승요금이며, contious feature 입니다. 한번 historgram을 그려보겠습니다.

In [357]:
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
g = sns.distplot(df_train['Fare'], color='b', label='Skewness : {:.2f}'.format(df_train['Fare'].skew()), ax=ax)
g = g.legend(loc='best')

* 보시다시피, distribution이 매우 비대칭인 것을 알 수 있습니다.(high skewness). 만약 이대로 모델에 넣어준다면 자칫 모델이 잘못 학습할 수도 있습니다. 몇개 없는 outlier에 대해서 너무 민감하게 반응한다면, 실제 예측 시에 좋지 못한 결과를 부를 수 있습니다.
* outlier의 영향을 줄이기 위해 Fare에 log를 취하겠습니다. 
* 여기서 우리는 pandas의 유용한 기능을 사용할 겁니다. dataFrame의 특정 columns에 공통된 작업(함수)를 적용하고 싶으면 아래의 map, 도는 apply를 사용하면 매우 쉽게 적용할 수 있습니다.
* 우리가 지금 원하는 것은 Fare columns 의 데이터 모두를 log값 취하는 것인데, 파이썬의 간단한 lambda 함수를 이용해 간단한 로그를 적용하는 map에 인수로 넣어주면, Fare columns 데이터에 그대로 적용이 됩니다. 매우 유용한 기능이니 꼭 숙지하세요!

In [358]:
# 아래 줄은 뒤늦게 발견하였습니다. 13번쨰 강의에 언급되니, 일단 따라치시고 넘어가면 됩니다.
df_test.loc[df_test.Fare.isnull(),'Fare']=df_test['Fare'].mean() # testset 에 있는 nan value를 평균값으로 치환합니다.

df_train['Fare'] = df_train['Fare'].map(lambda i: np.log(i) if i > 0 else 0)
df_test['Fare'] = df_test['Fare'].map(lambda i: np.log(i) if i > 0 else 0)

In [359]:
fig, ax = plt.subplots(1, 1, figsize=(8, 8))
g = sns.distplot(df_train['Fare'],color='b', label='Skewness : {:.2f}'.format(df_train['Fare'].skew()), ax=ax) # distplot은 distribuiton plot이라고 생각하면 될거 같다. 분포그래프.
g = g.legend(loc='best')

* log를 취하니, 이제 비대칭성이 많이 사리진 것을 볼 수 있습니다.
* 우리는 이런 작업을 사용해 모델이 좀 더 좋은 성능을 내도록 할 수 있습니다.
* 사실 방금한 것은 feature engineering 에 들어가는 부분인데, 여기서 작업했습니다.
* 모델을 학습시키기 위해, 그리고 그 모델의 성능을 높이기 위해 feature 들에 여러조작을 가하거나, 새로운 feature를 추가하는 것을 feature engineering이라고 하는데, 우리는 이제 그것을 살펴볼 것입니다.

# 2.9 Cabin
*   이 features는 NaN이 대략 80%이므로, 생존에 영향을 미칠 중요한 정보를 얻어내기가 쉽지는 않습니다.
*   그러므로 우리가 세우려는 모델에 포함시키지 않도록 하겠습니다.

In [360]:
df_train.head()

# 2.10 Ticket
*    이 feature는 NaN은 없습니다. 일단 string data이므로 우리가 어떤 작업들을 해주어야 실제 모델에 사용할 수 있는데, 이를 위해선 사실 아이디어가 필요합니다.

In [361]:
df_train['Ticket'].value_counts()

* 보시다시피, ticket number는 매우 다양합니다. 우리는 여기서 어떤 특징을 이끌어내서 생존과 연관시킬 수 있을까요?


# 3. Feature engineering
*    본격적인 feature engineering을 시작해보겠습니다.
*    가장먼저, dataset 에 존재하는 null data를 채우려고 합니다
*    아무 숫자로 채울 수는 없고, null data를 포한하는 feature의 statistics 를 참고하거나, 다른 아이디어를 짜내어 채울 수 있습니다.
*    null data 를 어떻게 채우느냐에 따라 모델의 성능이 좌지우지될수 있기 때문에, 신경써줘야할 부분입니다.
*    Feature engineering은 실제 모델의 학습에 쓰려고 하는 것이므로, train 뿐만 아니라 test도 똑같이 적용해줘야 합니다. 잊지맙시다

## 3.1 Fill Null
### 3.1.1 Fill Null in Age using title
* Age에는 null data가 177개나 있습니다. 이를 채울 수 있는 여러 아이디어가 있을 것인데, 여기서 우리는 title + statistics를 사용해 보겠습니다.
* 영어에는 Miss, Mrs 같은 title이 존재합니다. 각 탑승갱의 이름에는 꼭 이런 title이 들어가게 되는데 이를 사용해보겠습니다.
* pandas series 에는 data를 string으로 바꿔주는 str method, 거기에 정규표현식을 적용하게 해주는 extract method가 있습니다. 이를 사용하여 title을 쉽게 추출할 수 있습니다. title을 Initial column에 저장하겠습니다.

In [362]:
df_train['Initial']=df_train.Name.str.extract('([A-Za-z]+)\.') # lets extract the Salutations

df_test['Initial']=df_test.Name.str.extract('([A-Za-z]+)\.') # lets extract the Salutations

* pandas 의 crosstab을 이용하여 우리가 추출한 Initial과 Sex간의 count를 살펴봅시다.

In [363]:
pd.crosstab(df_train['Initial'], df_train['Sex']).T.style.background_gradient(cmap='summer_r') #Checking the Initials with the Sex

* 위 table을 참고하여, 남자, 여자가 쓰는 initial을 구분해 보겠습니다. replace 메소드를 사용하면, 특정 데이터 값을 원하는 값으로 치환해줍니다.

In [364]:
df_train['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don', 'Dona'],
                        ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr', 'Mr'],inplace=True)

df_test['Initial'].replace(['Mlle','Mme','Ms','Dr','Major','Lady','Countess','Jonkheer','Col','Rev','Capt','Sir','Don', 'Dona'],
                        ['Miss','Miss','Miss','Mr','Mr','Mrs','Mrs','Other','Other','Other','Mr','Mr','Mr', 'Mr'],inplace=True)

In [365]:
df_train.groupby('Initial').mean()

* 여성과 관계있는 Miss, Mr, Mrs 가 생존률이 높은 것을 볼 수 있습니다.

In [366]:
df_train.groupby('Initial')['Survived'].mean().plot.bar()

* 이제 본격적으로 Null 을 채울 것입니다. null data를 채우는 방법은 정말 많이 존재합니다. statistics 를 활용하는 방법도 있고, null data가 없는 데이터를 기반으로 새로운 머신러닝 알고리즘을 만들어 예측해서 채워넣는 방식도 있습니다. 여기서는 statistics를 활용하는 방법을 사용할 것입니다.
* 여기서 statistics 는 train data의 것을 의미합니다. 우리는 언제나 test를 unseen으로 둔 상태로 놔둬야 하며, train에서 얻은 statisics를 기반으로 testdml null data를 채워줘야 합니다.

In [367]:
df_train.groupby('Initial').mean()

* Age의 평균을 이용해 Null value를 채우도록 하겠습니다.
* pandas dataframe 을 다룰 때에는 booltean array를 이용해 indexing 하는 방법이 참으로 편리합니다.
* 아래 코드 첫줄을 해석하면, isnull() 이면서 Initial이 Mr인 조건을 만족하는 row(탑승객)의 'Age'의 값을 33으로 치환한다 입니다.
* loc + boolean + column 을 사용해 값을 치환하는 방법은 자주 쓰이므로 꼭 익숙해집시다.


In [368]:
df_train.loc[(df_train.Age.isnull())&(df_train.Initial=='Mr'),'Age'] = 33
df_train.loc[(df_train.Age.isnull())&(df_train.Initial=='Mrs'),'Age'] = 36
df_train.loc[(df_train.Age.isnull())&(df_train.Initial=='Master'),'Age'] = 5
df_train.loc[(df_train.Age.isnull())&(df_train.Initial=='Miss'),'Age'] = 22
df_train.loc[(df_train.Age.isnull())&(df_train.Initial=='Other'),'Age'] = 46

df_test.loc[(df_test.Age.isnull())&(df_test.Initial=='Mr'),'Age'] = 33
df_test.loc[(df_test.Age.isnull())&(df_test.Initial=='Mrs'),'Age'] = 36
df_test.loc[(df_test.Age.isnull())&(df_test.Initial=='Master'),'Age'] = 5
df_test.loc[(df_test.Age.isnull())&(df_test.Initial=='Miss'),'Age'] = 22
df_test.loc[(df_test.Age.isnull())&(df_test.Initial=='Other'),'Age'] = 46

* 여기선 간단하게 Null을 채웠지만, 좀 더 다양한 방법을 쓴 예시들이 다른 커널에 존재합니다.

## 3.1.2 Fill Null in Embarked

In [369]:
print('Embarked has ', sum(df_train['Embarked'].isnull()), 'Null values')

* Embarked 는 Null value가 2개이고, S에서 가장 많은 탑승객이 있었으므로, 간단하게 Null을 S로 채우겠습니다.
* dataframe 의 fillna method를 이용하면 쉽게 채울 수 있습니다. 여기서 inplace=True 로 하면 df_train 에 fillna를 실제로 적용하게 됩니다.

In [370]:
df_train['Embarked'].fillna('S', inplace=True)

## 3.2 Change Age(continuous to categorical)
*    Age는 현재 continuous feature입니다. 이대로 써도 모델을 세울 수 있지만, Age를 몇개 group으로 나누어 category화 시켜줄 수 도 있습니다. continuous를 categorical로 바꾸면 자칫 information loss가 생길 수도 있습니다만, 본 튜토리얼에서는 다양한 방법틀 소개하는 것이 목적이므로 진행하도록 하겠습니다.
*    방법은 여러가지가 있습니다. ataframe의 Indexing방법인 loc를 사용하여 직접해줄 수 있고, 아니면 apply를 사용해 함수를 넣어줄 수 있습니다.
*    첫번째로 loc를 사용한 방법입니다. loc는 자주쓰게 되므로 그 사용법을 숙지하면 좋습니다.
*    나이는 10살 간격으로 나누겠습니다.

In [371]:
df_train['Age_cat'] = 0
df_train.loc[df_train['Age']<10,'Age_cat']=0
df_train.loc[(10<=df_train['Age']) & (df_train['Age']<20), 'Age_cat'] =1
df_train.loc[(20<=df_train['Age']) & (df_train['Age']<30), 'Age_cat'] =2
df_train.loc[(30<=df_train['Age']) & (df_train['Age']<40), 'Age_cat'] =3
df_train.loc[(40<=df_train['Age']) & (df_train['Age']<50), 'Age_cat'] =4
df_train.loc[(50<=df_train['Age']) & (df_train['Age']<60), 'Age_cat'] =5
df_train.loc[(60<=df_train['Age']) & (df_train['Age']<27), 'Age_cat'] =6
df_train.loc[70<=df_train['Age'], 'Age_cat'] = 7

df_test['Age_cat'] = 0
df_test.loc[df_test['Age']<10, 'Age_cat'] = 0
df_test.loc[(10<=df_test['Age']) & (df_test['Age'] < 20), 'Age_cat'] = 1
df_test.loc[(20<=df_test['Age']) & (df_test['Age'] < 30), 'Age_cat'] = 2
df_test.loc[(30<=df_test['Age']) & (df_test['Age'] < 40), 'Age_cat'] = 3
df_test.loc[(40<=df_test['Age']) & (df_test['Age'] < 50), 'Age_cat'] = 4
df_test.loc[(50<=df_test['Age']) & (df_test['Age'] < 60), 'Age_cat'] = 5
df_test.loc[(60<=df_test['Age']) & (df_test['Age'] < 27), 'Age_cat'] = 6
df_test.loc[70 <= df_test['Age'], 'Age_cat'] = 7

* 두번째로 간단한 함수를 만들어 apply 메소드에 넣어주는 방법입니다.
* 훨씬 수월합니다.

In [372]:
def category_age(x):
    if x<10:
        return 0
    elif x < 20:
        return 1
    elif x < 30:
        return 2
    elif x < 40:
        return 3
    elif x < 50:
        return 4
    elif x < 60:
        return 5
    elif x < 70:
        return 7
df_train['Age_cat_2'] = df_train['Age'].apply(category_age)

* 두가지 방법이 잘 적용됬다면, 둘다 같은 결과를 내야합니다.
* 이를 확인하기 위해 Series rks boolean 비교후 all() 메소드를 사용합시다. all() 메소드는 모든 값이 True면 True, 하나라도 False가 있으면 False를 줍니다.

In [373]:
print('1번방법, 2번방법 둘다 같은 결과를 내면 True wnjdigka ->', (df_train['Age_cat']==df_train['Age_cat_2']).all())

###### 1번방법, 2번 방법 둘다 같은 결과를 내면 True 줘야함 -> True

* 보시다시피 True 입니다. 둘 줄 편한 걸 선택하시면 되겠습니다.
* 이제 중복되는 Sge_cat 컬럼과 원래 컬럼 Age를 제거하겠습니다.

In [374]:
df_train.drop(['Age', 'Age_cat_2'],axis=1, inplace=True)
df_test.drop(['Age'], axis=1, inplace=True)

# 3.3 Change Initial, Embarked and Sex (string to numerical)
* 현재 Initial은 Mr, Mrs, Miss, Master, Other 총 5개로 이루어져 있스빈다. 이런 카테고리로 표현되어져 있는 데이터를 모델에 인풋으로 넣어줄 때 우리가 해야할 것은 먼저 컴퓨터가 인식할 수 있도록 수치화 시켜야 합니다.
* map method를 가지고 간단히 할 수 있습니다.
* 사전 순서대로 정리하여 mapping 하겠습니다

In [375]:
df_train['Initial'] = df_train['Initial'].map({'Master': 0, 'Miss': 1, 'Mr': 2, 'Mrs':3, 'Other': 4})
df_test['Initial'] = df_test['Initial'].map({'Master':0, 'Miss': 1, 'Mr':2, 'Mrs':3, 'Other':4})

* Embarked 도 C,Q,S로 이루어져 있습니다. map을 이용해 바꿔봅시다.
* 그러기 앞서서, 특정 column에 어떤 값들이 있는 지 확인해보는 방법을 잠깐 살펴보겠습니다. 간단히 unique() 메소드를 쓰거나, value_counts()를 써서 count 까지 보는 방법이 있습니다.

In [376]:
df_train['Embarked'].unique()

In [377]:
df_train['Embarked'].value_counts()

* 위 두 방법을 사용해 Embarked가 S,C,Q 세가지로 이루어진 것을 볼 수 있스빈다. 이제 map을 사용해봅시다

In [378]:
df_train['Embarked'] = df_train['Embarked'].map({'C':0, 'Q':1,'S':2})
df_test['Embarked'] = df_test['Embarked'].map({'C':0,  'Q':1, 'S':2})

* 한번 Null이 사라졌는지 확인해봅시다. Embarked Column만 가져온 것은 하나의 Pandas의 Series객체므로, isnull()메소드를 사용해 Series의 값들이 null인지 아닌지에 대해 boolean 값을 얻을 수 있습니다. 그리고 이것에 any()를 사용하여, True가 단하나라도 있을시 ( Null이 한개라도 있을 시) Ture를 반환해주게 됩니다. 우리는 Null을 S로 다 바꿔주었으므로 False를 얻게 됩니다

In [379]:
df_train['Embarked'].isnull().any()

*Sex도 Female, male 로 이어져 있습니다. map을 이용해 바꿔봅시다.

In [380]:
df_train['Sex'] = df_train['Sex'].map({'female': 0, 'male': 1})
df_test['Sex'] = df_test['Sex'].map({'female': 0,'male':1})

* 여지껏 고생하셨습니다. 이제 각 Freature 간의 상관관계를 한번 보려고 합니다. 두 변수간의 Pearson correlation을 구하면 (-1, 1) 사이의 값을 얻을 수 있습니다. -1로 갈수록 음의 상관관계, 1로 갈수록 양의 상관관계를 의미하며, 0은 상관관계가 없다는 것을 의미합니다. 구하는 수식은 아래와 같습니다.

$$r_{xy} = \frac{Cov(x,y)}{S_{x}S_{y}} = \frac{\frac{1}{n-1}\sum_{i=1}^n(x_{i}-\bar x)(y_{i}-\bar y)}{S_{x}S_{y}}$$


* 우리는 여러 feature를 가지고 있으니 이를 하나의 maxtrix형태로 보면 편할 텐데, 이를 heatmap plot 이라고 하며, dataframe의 corr()메소드와 seaborn을 가지고 편하게 그릴 수 있습니다.

In [381]:
heatmap_data = df_train[['Survived', 'Pclass', 'Sex', 'Fare', 'Embarked', 'FamilySize', 'Initial', 'Age_cat']] 

colormap = plt.cm.RdBu #plt.cm.RdBu는 나중에 정의된 것 처럼 컬러맵을 나타냄
plt.figure(figsize=(14, 12))
plt.title('Pearson Correlation of Features', y=1.05, size=15)
sns.heatmap(heatmap_data.astype(float).corr(), linewidths=0.1, vmax=1.0,
           square=True, cmap=colormap, linecolor='white', annot=True, annot_kws={"size": 16})

del heatmap_data

* 우리가 EDA에서 살펴봣듯이, Sex와 Pclass가 Survived에 상관관계가 어느 정도 있음을 볼 수 있습니다.
* 생각보다 fare 와 Embarked 도 상관관계가 있음을 볼 수 있습니다.
* 또한 우리가 여기서 얻을 수 있는 정보는 서로 강한 상관관계를 가지는 feature들이 없다는 것입니다.
* 이것은 우리가 모델을 학습시킬 떄, 불필요한(redundant, superfluous) feature가 없다는 것을 의미합니다. 1 또는 -1의 상관관계를 가진 feature A,B가 있다면, 우리가 얻을 수 있는 정보는 사실 하나일 거니까요.
* 이제 실제로 모델을 학습시키기 앞서서 data preprocessing(전처리)을 진행해보겠습니다. 

# 3.4 One-hot encoding on Initial and Embarked

* 수치화시킨 카테고리 데이터를 그대로 넣어도 되지만, 모델의 성능을 높이기 위해 one-hot encoding을 해줄 수 있습니다
* 수치화는 간단히 Master ==0, Miss ==1, Mr ==2, Mrs ==3, Other ==4로 매핑해주는 것을 말합니다.
* One-hot encoding은 위 카테고리를 아래와 같이 (0,1)로 이루어진 5차원의 벡터로 나타내는 것을 말합니다.
* 위와 같은 작업을 직접 코딩할 수도 있지만, pandas의 get_dummiesfmf 사용하여 쉽게 해결할 수 있습니다.
* 총 5개의 카테고리니, one-hot encoding을 하고 나면 새로운 5개의 column이 생겨납니다. 
* Initial을 prefix로 두어서 구분이 쉽게 만들어 줍니다.

In [382]:
df_train = pd.get_dummies(df_train, columns = ['Initial'], prefix='Initial')
df_test = pd.get_dummies(df_test, columns=['Initial'], prefix='Initial')

In [383]:
df_train.head()

* 보시다시피 오른쪽에 우리가 만들려고 했던 One-hot-encoded columns가 생성된 것이 보입니다.
* Embarked 에도 적용하겠습니다. Initial 떄와 마찬가지로 one-hot encoding을 사용해 표현하겠습니다.

In [384]:
df_train = pd.get_dummies(df_train, columns=['Embarked'], prefix='Embarked')
df_test = pd.get_dummies(df_test, columns=['Embarked'], prefix='Embarked')

* 아주 쉽게 one-hot encoding을 적용했습니다.
* sklearn로 Labelencodere + OneHotencoder 이용해도 one-hot encoding이 가능합니다
* 다른 튜토리얼을 한번 써보겠습니다. 여기서는 get_dummies 로 충분히 가능하기 때문에 get_dummies 만으로 끝내겠습니다.
* 가끔 category가 100개가 넘어가는 경우가 있습니다. 이때 one-hot encoding을 사용하면 column이 100개가 생겨, 학습 시 버거울 경우가 있습니다. 이런 경우는 다른방법을 사용하기도 하는데, 이는 다음에 한번 다뤄보겠습니다.

# 3.5 Drop columns

* 필요한 columns만 남기고 다 지웁시다

In [385]:
df_train.drop(['PassengerId','Name', 'SibSp', 'Parch', 'Ticket', 'Cabin'], axis=1, inplace=True)
df_test.drop(['PassengerId', 'Name', 'SibSp', 'Parch', 'Ticket', 'Cabin'], axis=1, inplace=True)

In [386]:
df_train.head()

In [387]:
df_test.head()

* 보시다시피, train의 Survived feature(tager class)을 뺴면 train, test 둘다 같은 columns를 가진 걸 확인할 수 있습니다.

# 4. Building Machin learning model and prediciton using the trained model

* 이제 준비가 다 되었으니 sklearn을 사용해 본격적을 머신러닝 모델을 만들어 봅시다.

In [388]:
#importin all the required ML packages
from sklearn.ensemble import RandomForestClassifier # 유명한 randomforestclassfier 입니다.
from sklearn import metrics # 모델의 평가를 위해서 씁니다
from sklearn.model_selection import train_test_split # traning set을 쉽게 나눠주는 함수입니다.

* Sklearn은 머신러닝의 처음부터 끝까지가 다 있습니다. feature engineering, preprocessing, 지도 학습 알고지름, 비지도 학습 알고리즘, 모델 명가, 파이프라인 등 머신러닝에 관련된 모든 작업들이 손쉬운 인터페이스로 구현되어 있습니다. 데이터 분석 + 머신러닝을 하고 싶다면, 이 라이브러리는 반드시 숙지해야합니다.
* 파이썬 라이브러리를 활용한 머신러닝 책을 사서 공부하시길 매우 추천
* 지금 타이타닉 문제는 target class(survived)가 있으며, target class는 0,1 로 이루어져 있으므로 (binary) binary classfication문제입니다.
* 우리가 지금 가지고 있는 train set 의 survived를 제외한 input을 자긴고 모델을 최적화시켜서 각 샘플(탑승객)의 생존유무를 판단하는 모델을 만들어 냅니다.
* 그후 모델이 학습하지 않았던 test set 을 input으로 주어서 test set 의 각 샘플(탑승객)의 생존 유무를 예측해봅니다.

# 4.1 Preparation - Split dataset into train, valid, test set
*    가장먼저, 학습에 쓰일 데이터와 , target laber(Survived)를 분리합니다. drop을 사용해 간단히 할 수 있습니다.

In [389]:
X_train = df_train.drop('Survived', axis=1).values
target_label = df_train['Survived'].values
X_test = df_test.values

* 보통 train, test 만 언급되지만, 실제 좋은 모델을 만들기 위해서 우리는 valid set을 따로 만들어 모델평가를 해봅니다
* 마치 축구대표팀이 팀훈련(train)을 하고 바로 월드컵(test)로 나가는 것이 아니라, 팀훈련(train)을 한 다음 평가전(valid)를 거쳐 팀 훈련 정도(학습정도)를 확인하고 월드컵(test)에 나가는 것과 비슷합니다.
* trian_test_split을 사용하여 쉽게 train 셋을 분리할 수 있습니다.

In [390]:
X_tr, X_vld, y_tr, y_vld = train_test_split(X_train, target_label, test_size=0.3, random_state=2018)

* sklearn 에서는 여러 머신러닝 알고리즘을 지원해줍니다. 열거하기엔 너무 많으므로, 직접 documentation에 들어가 보시길 추천합니다. http://scikit-learn.org/stable/supervised_learning.html#supervised-learning 여기에 들어가시면 지원되는 알고리즘 수에 놀라실 겁니다.
* 본 튜토리얼에서는 랜덤포레스트 모델을 사용하도록 하겠습니다. 
* 랜덤포레스트는 결정트리기반 모델이며, 여러 결정 트리들을 앙상블한 모델입니다. 더 구체적인 모델 설명은 여러 블로그들 참고하시면 될 것이고, 저도 한번 추후 다뤄보겠습니다.
* 각 머신러닝 알고리즘에는 여러 파라미터들이 있습니다. 랜덤포레스트분류기도 n_estimators, max_features, max_depth, min_samples_split, min_samples_leaf등 여러 파라미터들이 존재합니다. 이것들이 어떻게 세팅되냐에 따라 같은 데이터셋이라 하더라도 모델의 성능이 달라집니다.
* 파라미터 튜닝은 시간, 경험, 알고리즘에 대한 이해 등이 필요합니다. 결국 ㅁ낳이 써봐야 모델도 잘 세울 수 있는 것이죠. 그래서 캐글을 추천합니다. 여러 데이터셋을 가지고 모델을 이리저리 써봐야 튜닝하는 감이 생길테니까요!
* 일단 지금은 튜토리얼이니 파라미터 튜닝은 잠시 제쳐두기로 하고, 기본 default 세팅으로 진행하겠습니다.
* 모델 객체를 만들고, fit 메소드로 학습시킵니다.
* 그런 후 valid set input을 넣어주어 예측값(x_vldsample(탑승객)의 생존여부)를 얻습니다.

# 4.2 Model generation and prediciton

In [391]:
model = RandomForestClassifier()
model.fit(X_tr, y_tr)
prediction = model.predict(X_vld)

* 단 세줄만으로 여러분은 모델을 세우고, 예측까지 해봤습니다.
* 자 이제 모델의 성능을 한번 보겠습니다.

In [392]:
print('총 {}명 중 {:.2f}%정확도로 생존을 맞춤'.format(y_vld.shape[0], 100*metrics.accuracy_score(prediction, y_vld)))

# 4.3 Feature importance
* 학습된 모델은 feature importance를 가지게 되는데, 우리는 이것을 확인하여 지금 만든 모델이 어떤 feature에 영향을 많이 받았는지 확인할 수 있습니다.
* 쉽게 말게, 10 = 4x1 + 2x2 + 1*x3을 생각하면, 우리는 x1이 결과값(10)에 큰 영향을 준다고 생각 할 수 있습니다. feature importance는 4,2,1을 이야기하며, x1이 가장 큰 값(4)를 가지므로, 이모델에 가장 큰 영향을 미친다고 말할 수 있습니다.
* 학습된 모델은 기본적으로 featureimportances 를 가지고 있어서 쉽게 그 수치를 얻을 수 있습니다.
* pandas series 를 이용하면 쉽게 soritng 을 하여 그래프를 그릴 수 있습니다.

In [393]:
from pandas import Series

feature_importance = model.feature_importances_
Series_feat_imp = Series(feature_importance, index=df_test.columns)

In [394]:
plt.figure(figsize=(8, 8))
Series_feat_imp.sort_values(ascending=True).plot.barh()
plt.xlabel('Feature importance')
plt.ylabel('Feature')
plt.show()

* 우리가 얻은 모델에서는 Fare 가 가장 큰 영향력을 가지며, 그 뒤로 Initial_2, Age_cat,Pclass가 차례로 중요도를 가집니다
* 사실 feature importance 는 지금 모델에서의 importance를 나타냅니다. 만약 다른 모델을 사용하게 된다면 feature importance가 다르게 나올 수 있습니다.
* 이 feature importance를 보고 Fare가 중요한 feature 일 수 있다고 판단을 내릴 수는 있지만, 이것은 결국 모델에 귀속되는 하나의 결론이므로 통계적으로 좀 더 살펴보긴 해야합니다.
* feature importance를 가지고 좀 더 정확도가 높은 모델을 얻기 위해 feature selection을 할 수 도 있고, 좀 더 빠른 모델을 얻기위해 feature 제거를 할 수 있습니다.

# 4.4 Prediction on Test set
* 이제 모델이 학습하지 않았던(보지 않았던) 테스트셋을 모델에 주어서, 생존여부를 예측해보겠습니다.
* 이 결과는 실제로 submission(제출용) 이므로 결과는 leaderboard에서 확인할 수 있습니다.
* 캐글에서 준 파일, gender_submission.csv 파일을 읽어서 제출 준비를 하겠습니다.

In [395]:
submission = pd.read_csv('../input/titanic/gender_submission.csv')
submission.head()

이제 testset에 대해여 예측을 하고, 그 결과를 csv 파일로 저장해보겠습니다.

In [396]:
prediction = model.predict(X_test)
submission['Survived'] = prediction

In [397]:
submission.to_csv('./my_first_submission.csv', index=False)